In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm 

In [2]:
# Var : Variable
# Relas : Relational (sama dengan,tidak sama dengan,lebih dari, kurang dari, lebih dari sama dengan, kurang dari sama dengan)
# Gate : Logical Gate

TRAIN_DATA = [
    ('Total Buah lebih dari 15', { #lebih dari
        'entities': [(11, 21, 'Relas')]
    }),
     ('Cable Type sama dengan Optic', { #sama dengan
        'entities': [(11, 22, 'Relas')]
    }),
    ('Tanggal kurang dari 17', { #kurang dari
        'entities': [(8, 19, 'Relas')]
    }),
    ('Username sama dengan benar dan Password sama dengan benar', { #sama dengan, dan, sama dengan
        'entities': [(9, 20, 'Relas'), (27, 30, 'Gate'), (40, 51, 'Relas')]
    }),
    ('Harga lebih dari 1000 dan Total kurang dari 2000', {
        'entities': [(6, 16, 'Relas'), (22, 25, 'Gate'), (32, 43, 'Relas')] #lebih dari, dan, kurang dari
    }),
    ('Rating lebih dari sama dengan 4 dan Harga kurang dari 500', { #lebih dari sama dengan, dan, kurang dari
        'entities': [(7, 29, 'Relas'), (32, 35, 'Gate'), (42, 53, 'Relas')]
    }),
    ('Suhu sama dengan 25 atau Kelembaban lebih dari 70', { #sama dengan, atau, lebih dari
        'entities': [(5, 16, 'Relas'), (20, 24, 'Gate'), (36, 46, 'Relas')]
    }),
    ('Juara sama dengan 1', { #sama dengan
        'entities': [(6, 17, 'Relas')]
    }),
    ('Jarak lebih dari sama dengan 300 meter dan Kecepatan lebih dari 25', { #lebih dari sama dengan, dan, lebih dari
        'entities': [(6, 28, 'Relas'), (39, 42, 'Gate'), (53, 63, 'Relas')]
    }),
    ('Berat tidak sama dengan 100 kg', { #tidak sama dengan
        'entities': [(6, 23, 'Relas')]
    }),
    ('Berat roda lebih dari sama dengan 100 kg dan bobot kendaraan kurang dari 1 ton', { #lebih dari sama dengan, dan, kurang dari
        'entities': [(11, 33, 'Relas'), (41, 44, 'Gate'), (61, 72, 'Relas')]
    }),
    ('Harga apel tidak sama dengan harga jeruk',{
        'entities': [(11, 28, 'Relas')]
    }),
    ('Rata-rata usia lebih dari sama dengan 25 tahun',{
        'entities': [(15, 37, 'Relas')]
    }),
    ('Jumlah buku lebih dari 10 ribu dan jenis buku sama dengan novel',{
        'entities': [(12, 22, 'Relas'), (31, 34, 'Gate'), (46, 57, 'Relas')]
    }),
    ('Tinggi pohon lebih dari 3 meter dan jumlah cabang sama dengan 20',{
        'entities': [(13, 23, 'Relas'), (32, 35, 'Gate'), (50, 61, 'Relas')]
    }),
    ('Harga tiket masuk kurang dari 50 ribu rupiah atau anak-anak lebih dari sama dengan 5 tahun',{
        'entities': [(18, 29, 'Relas'), (45, 49, 'Gate'), (60, 82, 'Relas')]
    }),
    ('Jarak tempuh kurang dari sama dengan 10 kilometer dan waktu tempuh kurang dari 30 menit',{
        'entities': [(13, 36, 'Relas'), (50, 57, 'Gate'), (67, 78, 'Relas')]
    }),
    ('Suhu pegunungan kurang dari 15 derajat Celsius atau cuaca sama dengan bersalju dan suhu kurang dari 0 derajat Celsius',{
        'entities': [(16, 27, 'Relas'), (47, 52, 'Gate'), (58, 69, 'Relas'), (79, 82, 'Gate'), (88, 96, 'Relas')] 
    }),
    ('Jumlah penonton lebih dari 5000 atau tiket VIP kurang dari sama dengan 100',{
        'entities': [(16, 28, 'Relas'), (32, 36, 'Gate'), (47, 70, 'Relas')]
    }),
    ('Cuaca hari ini tidak sama dengan kemarin',{
        'entities': [(15, 32, 'Relas')]
    }),
    ('Jumlah penumpang sama dengan 40 orang',{
        'entities': [(17, 28, 'Relas')]
    }),
    ('Jumlah pohon lebih dari sama dengan 200 dan jenisnya tidak sama dengan pohon cemara',{
        'entities': [(13, 35, 'Relas'), (40, 43, 'Gate'), (53, 70, 'Relas')]
    }),
    ('Suhu air lebih dari sama dengan 25 derajat Celsius dan suhu ruangan kurang dari 30 derajat Celsius',{
        'entities': [(9, 31, 'Relas'), (51, 54, 'Gate'), (68, 79, 'Relas')]
    }),
    ('Harga minyak goreng kurang dari 20 ribu rupiah dan harga telur lebih dari 15 ribu rupiah',{
        'entities': [(20, 31, 'Relas'), (47, 50, 'Gate'), (63, 73, 'Relas')]
    }),
    ('Harga set alat masak sama dengan 150 ribu',{
        'entities': [(21, 32, 'Relas')]
    }),
    ('Jumlah kursi sama dengan 50 dan kapasitas lebih dari 100 orang',{
        'entities': [(13, 24, 'Relas'), (28, 31, 'Gate'), (42, 52, 'Relas')]
    }),
    ('Harga kue kurang dari 10 ribu rupiah dan harga krim lebih dari 5 ribu rupiah',{
        'entities': [(10, 21, 'Relas'), (37, 40, 'Gate'), (52, 62, 'Relas')]
    }),
    ('Jarak tempuh lebih dari 100 kilometer',{
        'entities': [(13, 23, 'Relas')]
    }),
    ('Jumlah penumpang sama dengan 80 orang dan jumlah kursi kosong lebih dari 10',{
        'entities': [(17, 28, 'Relas'), (38, 41, 'Gate'), (62, 72, 'Relas')]
    }),
    ('Kapasitas RAM sama dengan 8 gigabyte',{
        'entities': [(14, 25, 'Relas')]
    }),
    ('Jumlah aplikasi sama dengan 50',{
        'entities': [(16, 27, 'Relas')]
    }),
    ('Kecepatan prosesor lebih dari 2.5 GHz dan jumlah memori RAM kurang dari 8 gigabyte',{
        'entities': [(19, 29, 'Relas'), (38, 41, 'Gate'), (60, 72, 'Relas')]
    }),
    ('Jumlah inti prosesor lebih dari 4 dan kecepatan clock kurang dari 2.0 GHz',{
        'entities': [(21, 31, 'Relas'), (34, 37, 'Gate'), (54, 65, 'Relas')]
    }),
    ('Kecepatan internet lebih dari 100 Mbps dan jumlah perangkat kurang dari 5',{
        'entities': [(19, 29, 'Relas'), (39, 42, 'Gate'), (60, 71, 'Relas')]
    }),
    ('Resolusi layar lebih dari 3840 x 2160 pixel dan refresh rate kurang dari 120 Hz',{
        'entities': [(15, 25, 'Relas'), (44, 47, 'Gate'), (61, 72, 'Relas')]
    }),
    ('Usia baterai lebih dari sama dengan 5 tahun dan waktu pengisian daya kurang dari 2 jam',{
        'entities': [(13, 35, 'Relas'), (44, 47, 'Gate'), (69, 80, 'Relas')]
    }),
    ('Kecepatan transfer data ini lebih dari 100 MBps dan kapasitas penyimpanan kurang dari 64 gigabyte',{
        'entities': [(28, 38, 'Relas'), (48, 51, 'Gate'), (74, 85, 'Relas')]
    }),

]

In [3]:
model = None
output_dir=Path(r"C:\Users\prade\OneDrive\Documents\Materi Kuliah\Sitasi Proposal\FINAL PROPOSAL\Node-Flask-Api\MODEL")
n_iter=100

In [4]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


In [5]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

ner

In [6]:
from spacy.training.example import Example

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

correct_train_entities = 0  # Untuk menghitung entitas yang diidentifikasi dengan benar pada data pelatihan
total_train_entities = 0    # Untuk menghitung total entitas sebenarnya pada data pelatihan

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
            
         # Hitung akurasi selama pelatihan
        doc = nlp(text)
        for ent in annotations["entities"]:
            start, end, label = ent
            if (start, end, label) in [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]:
                correct_train_entities += 1
            total_train_entities += 1
        
        accuracy_train = (correct_train_entities / total_train_entities) * 100
        print(f"Iterasi {itn + 1}, Train Data Accuracy: {accuracy_train:.2f}%")

        print(losses)


 32%|███▏      | 12/37 [00:00<00:00, 34.76it/s]c:\Users\prade\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Jumlah penonton lebih dari 5000 atau tiket VIP kur..." with entities "[(16, 28, 'Relas'), (32, 36, 'Gate'), (47, 70, 'Re...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
 43%|████▎     | 16/37 [00:00<00:00, 34.13it/s]c:\Users\prade\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Kecepatan prosesor lebih dari 2.5 GHz dan jumlah m..." with entities "[(19, 29, 'Relas'), (38, 41, 'Gate'), (60, 72, 'Re...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ig

Iterasi 1, Train Data Accuracy: 0.00%
{'ner': 193.97431032291934}


100%|██████████| 37/37 [00:01<00:00, 31.83it/s]


Iterasi 2, Train Data Accuracy: 50.00%
{'ner': 141.81173006436603}


100%|██████████| 37/37 [00:01<00:00, 35.00it/s]


...

Iterasi 98, Train Data Accuracy: 89.02%
{'ner': 0.9248104717313849}


100%|██████████| 37/37 [00:01<00:00, 31.28it/s]


Iterasi 99, Train Data Accuracy: 89.16%
{'ner': 2.012376524876434}


100%|██████████| 37/37 [00:01<00:00, 29.62it/s]

Iterasi 100, Train Data Accuracy: 89.29%
{'ner': 0.09522891789903344}


In [7]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob_) for t in doc])

Entities [('kurang dari sama dengan', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Tokens [('Jarak', '', 'O'), ('tempuh', '', 'O'), ('kurang', 'Relas', 'B'), ('dari', 'Relas', 'I'), ('sama', 'Relas', 'I'), ('dengan', 'Relas', 'I'), ('10', '', 'O'), ('kilometer', '', 'O'), ('dan', 'Gate', 'B'), ('waktu', '', 'O'), ('tempuh', '', 'O'), ('kurang', 'Relas', 'B'), ('dari', 'Relas', 'I'), ('30', '', 'O'), ('menit', '', 'O')]
Entities [('lebih dari sama dengan', 'Relas')]
Tokens [('Rata', '', 'O'), ('-', '', 'O'), ('rata', '', 'O'), ('usia', '', 'O'), ('lebih', 'Relas', 'B'), ('dari', 'Relas', 'I'), ('sama', 'Relas', 'I'), ('dengan', 'Relas', 'I'), ('25', '', 'O'), ('tahun', '', 'O')]
Entities [('sama dengan', 'Relas')]
Tokens [('Harga', '', 'O'), ('set', '', 'O'), ('alat', '', 'O'), ('masak', '', 'O'), ('sama', 'Relas', 'B'), ('dengan', 'Relas', 'I'), ('150', '', 'O'), ('ribu', '', 'O')]
Entities [('lebih dari', 'Relas')]
Tokens [('Total', '', 'O'), ('Buah', '', 'O'), ('lebih', 'Rela

In [8]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)    

Saved model to C:\Users\prade\OneDrive\Documents\Materi Kuliah\Sitasi Proposal\FINAL PROPOSAL\Node-Flask-Api\MODEL


In [9]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from C:\Users\prade\OneDrive\Documents\Materi Kuliah\Sitasi Proposal\FINAL PROPOSAL\Node-Flask-Api\MODEL
Entities [('kurang dari sama dengan', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas')]
Entities [('sama dengan', 'Relas')]
Entities [('lebih dari', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Entities [('sama dengan', 'Relas'), ('dan', 'Gate'), ('sama dengan', 'Relas')]
Entities [('sama dengan', 'Relas')]
Entities [('sama dengan', 'Relas')]
Entities [('lebih dari', 'Relas'), ('dan', 'Gate'), ('sama dengan', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas'), ('dan', 'Gate'), ('tidak sama dengan', 'Re

In [10]:
#TEST -----------------------------------------------------------

In [10]:
# Load the trained NER model
nlp = spacy.load(output_dir)

# Test text
test_text = "Total_buah sama dengan 17"

# Process the test text
doc = nlp(test_text)


# Print extracted entities
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob_) for t in doc])

Entities [('sama dengan', 'Relas')]
Tokens [('Total_buah', '', 'O'), ('sama', 'Relas', 'B'), ('dengan', 'Relas', 'I'), ('17', '', 'O')]


In [14]:
# Data tes
TEST_DATA = [
    ("Total buah sama dengan 17 dan besar bola kurang dari 17", {
        "entities": [(11, 22, "Relas"), (26, 29, "Gate"), (41, 52, "Relas")]
    }),
    ("Suara sama dengan 100 dan juara sama dengan menang", {
        "entities": [(6, 17, "Relas"), (22, 25, "Gate"), (32, 43, "Relas")]
    }),
    ("apel kurang dari 10 dan harga_beras lebih dari 200", {
        "entities": [(5, 16, "Relas"), (20, 23, "Gate"), (36, 46, "Relas")]
    })
    # Tambahkan data tes lainnya sesuai kebutuhan
]

correct_entities = 0  # Untuk menghitung entitas yang diidentifikasi dengan benar
total_entities = 0   # Untuk menghitung total entitas sebenarnya

for text, annotations in TEST_DATA:
    print(text)
    print(annotations)
    doc = nlp(text)  # Proses data tes dengan model
    print(doc.ents)

    for ent in doc.ents:
        print(ent.text, ent.label_)
    
    for ent in annotations["entities"]:
        start, end, label = ent
        if (start, end, label) in [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]:
            correct_entities += 1
        print(start, end, label)
        print(ent)
        total_entities += 1

accuracy = (correct_entities / total_entities) * 100
print(f"Accuracy: {accuracy:.2f}%")

Total buah sama dengan 17 dan besar bola kurang dari 17
{'entities': [(11, 22, 'Relas'), (26, 29, 'Gate'), (41, 52, 'Relas')]}
(sama dengan, dan, kurang dari)
sama dengan Relas
dan Gate
kurang dari Relas
11 22 Relas
(11, 22, 'Relas')
26 29 Gate
(26, 29, 'Gate')
41 52 Relas
(41, 52, 'Relas')
Suara sama dengan 100 dan juara sama dengan menang
{'entities': [(6, 17, 'Relas'), (22, 25, 'Gate'), (32, 43, 'Relas')]}
(sama dengan, dan, sama dengan)
sama dengan Relas
dan Gate
sama dengan Relas
6 17 Relas
(6, 17, 'Relas')
22 25 Gate
(22, 25, 'Gate')
32 43 Relas
(32, 43, 'Relas')
apel kurang dari 10 dan harga_beras lebih dari 200
{'entities': [(5, 16, 'Relas'), (20, 23, 'Gate'), (36, 46, 'Relas')]}
(kurang dari, dan, lebih dari)
kurang dari Relas
dan Gate
lebih dari Relas
5 16 Relas
(5, 16, 'Relas')
20 23 Gate
(20, 23, 'Gate')
36 46 Relas
(36, 46, 'Relas')
Accuracy: 100.00%
